In [21]:
#Import package
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib
from geopy.distance import geodesic
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
from datetime import datetime, timedelta, date
import network_analysis_module

In [3]:
# - Import data -
aed = pd.read_parquet('aed_locations.parquet.gzip')
postal_codes_subset = pd.read_csv('postal_codes.csv', sep=',')

In [4]:
# AED
print(aed.head())
aed_cl=aed.copy()

     id  type                address  number  postal_code municipality  \
0  13.0  None    Blvd. Fr. Roosevelt    24.0       7060.0     SOIGNIES   
1  70.0  None        Ch. De Wégimont    76.0       4630.0      Ayeneux   
2  71.0  None  Place Saint - Lambert     NaN       4020.0        Liège   
3  72.0  None          Rue du Doyard     NaN       4990.0     Lierneux   
4  73.0  None     Fond Saint Servais     NaN       4000.0        Liège   

  province location public available hours  
0  Hainaut     None      Y      None  None  
1    Liège     None   None      None  None  
2    Liège     None   None      None  None  
3    Liège     None   None      None  None  
4    Liège     None   None      None  None  


In [6]:
for column in aed_cl.columns:
    if aed_cl[column].dtype=='object':
        aed_cl[column]=aed_cl[column].str.title().str.strip()
    else:
        aed_cl[column]=aed_cl[column]    

aed_cl.loc[aed_cl['type'].isin(['Appareil fixe','Appareil Fixe','Appareil fixe-Vast apparaat','Vast apparaat','Vast apparaat']),'type']='Fixed'
aed_cl.loc[aed_cl['type'].isin(['Appareil Mobile- Mobiel apparaat','M5066A']),'type']='Mobile'

aed_cl['postal_code']=aed_cl['postal_code'].astype('str').str.replace('.0','')

for index, value in aed.iterrows():
    if aed_cl.iloc[index, 8] in ['J', 'Ja', 'Oui', 'Oui-Ja', 'y', 'Y']:
        aed_cl.loc[index, 'public'] = 'Yes'
    elif aed_cl.iloc[index, 8] in ['Non-Nee', 'N', 'Nee']:
        aed_cl.loc[index, 'public'] = 'No'
    else:
        aed_cl.iloc[index, 8] = 'None'
        
for index, value in aed.iterrows():
    if aed_cl.iloc[index, 9] in ['J', 'Ja', 'Oui', 'Oui-Ja', 'y', 'Y', 'oui via interphone','Accessible par toute personne présente dans l inrfastructure.','Oui-Ja "Niet tijdens activiteiten van de wielervrienden"']:
        aed_cl.loc[index, 'available'] = 'Yes'
    elif aed_cl.iloc[index, 9] in ['Non-Nee', 'N', 'Nee']:
        aed_cl.loc[index, 'available'] = 'No'
    elif aed_cl.iloc[index, 9] in ['09.00 - 17.00', '9.00 - 17.00', 'Nee']:
        aed_cl.loc[index, 'available'] = '09.00 - 17.00'
    elif aed_cl.iloc[index, 9] in ['09u-12u en op aanvraag']:
        aed_cl.loc[index, 'available'] = '09.00 - 12.00'
    elif aed_cl.iloc[index, 9] in ['5h00 à 25h00', 'de 5h00 à 25h00']:
        aed_cl.loc[index, 'available'] = '05.00 - 23.59'
    elif aed_cl.iloc[index, 9] in ['8:00 - 17:00', 'N', 'Nee']:
        aed_cl.loc[index, 'available'] = '8:00 - 17:00'
    elif aed_cl.iloc[index, 9] in ['16u-23u tijdens opening sporthal']:
        aed_cl.loc[index, 'available'] = '16.00 - 23.00'
    elif aed_cl.iloc[index, 9] in ['De 5h30 à 21h30']:
        aed_cl.loc[index, 'available'] = '05.30 - 21.30'
    elif aed_cl.iloc[index, 9] in ['Dispo 24/7 - sauf samedi de 12h à dimanche 20h', 'Dispo. tout le temps sauf du samedi midi au dimanche 21h']:
        aed_cl.loc[index, 'available'] = '00.00 - 23.59'
    elif aed_cl.iloc[index, 9] in ['Heure de bureau en semaine', 'Heures de bureau', 'Horaire d ouverture de la buvette','Pendant heures d ouverture du site',
                                   'Pendant les heures de cours','Tijdens de kantooruren','Tijdens kantooruren','Tijdens openingsuren van het museum','enkel tijdens kantooruren',
                                   'enkel tijdens de kantooruren (8 - 19u)','horaire d ouverture de la pharmacie','indien de site open is, volledig toegangkelijk','kantooruren',
                                   'selon heures d ouverture d Euro-Délices','tijdens de kantooruren','tijdens de kantooruren in de week','tijdens de openingsuren','tijdens de werkuren',
                                   'tijdens onze openingsuren, zie nr. 12','tijdens openinguren sportcentrum','tijdens werkuren','zie rooster','Heures de bureau ']:
        aed_cl.loc[index, 'available'] = 'Working Opening Time'
    elif aed_cl.iloc[index, 9] in ['maandag, dinsdag, donderdag, vrijdag', 'du lundi au vendredi ']: 
        aed_cl.loc[index, 'available'] = 'During Weekdays'
    else:
        aed_cl.iloc[index, 9] = 'None'  



     id  type                address  number  postal_code municipality  \
0  13.0  None    Blvd. Fr. Roosevelt    24.0       7060.0     SOIGNIES   
1  70.0  None        Ch. De Wégimont    76.0       4630.0      Ayeneux   
2  71.0  None  Place Saint - Lambert     NaN       4020.0        Liège   
3  72.0  None          Rue du Doyard     NaN       4990.0     Lierneux   
4  73.0  None     Fond Saint Servais     NaN       4000.0        Liège   

  province location public available hours  
0  Hainaut     None      Y      None  None  
1    Liège     None   None      None  None  
2    Liège     None   None      None  None  
3    Liège     None   None      None  None  
4    Liège     None   None      None  None  


In [9]:
aed_cl.dtypes
aed_cl['postal_code'].iloc[0]

'7060'

In [12]:
print(aed_cl['postal_code'].head(100))

0     7060
1     4630
2     4020
3     4990
4     4000
      ... 
95    1160
96    1200
97    1070
98    3290
99    3980
Name: postal_code, Length: 100, dtype: object


In [13]:
aed_cl['postal_code'] = aed_cl.apply(lambda x: float(x['postal_code']), axis=1)

In [14]:
aed_all = pd.merge(aed_cl,postal_codes_subset, left_on = 'postal_code', right_on = 'postcode', how ='left')
for a, b in zip(aed_all.columns, range(0,len(aed_all.columns))):
                print(f'{a}, column number {b}')

aed_subset = aed_all.iloc[:,[0,1,2,3,17,18,19,20,1,4,8,9,10]]
aed_subset.head()

id, column number 0
type, column number 1
address, column number 2
number, column number 3
postal_code, column number 4
municipality, column number 5
province, column number 6
location, column number 7
public, column number 8
available, column number 9
hours, column number 10
postcode, column number 11
smun_code, column number 12
reg_code, column number 13
prov_code, column number 14
arr_code, column number 15
mun_code, column number 16
Region, column number 17
Province, column number 18
Arrondissement, column number 19
Municipality, column number 20


,id,type,address,number,Region,Province,Arrondissement,Municipality,type,postal_code,public,available,hours
0,13.0,None,Blvd. Fr. Roosevelt,24.0,Wallonia,Hainaut (le),Soignies,Soignies,None,7060.0,Yes,None,None
1,70.0,None,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,None,4630.0,None,None,None
2,70.0,None,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,None,4630.0,None,None,None
3,71.0,None,Place Saint - Lambert,NaN,Wallonia,Liège,Liège,Liège,None,4020.0,None,None,None
4,72.0,None,Rue Du Doyard,NaN,Wallonia,Liège,Verviers,Lierneux,None,4990.0,None,None,None


In [15]:
# postal_codes_subset.to_csv('postal_codes.csv', index=False)
aed_subset.to_csv('aed.csv', index=False)

In [16]:
aed_df = pd.read_csv('aed.csv', sep=',')
aed_df.head()

,id,type,address,number,Region,Province,Arrondissement,Municipality,type.1,postal_code,public,available,hours
0,13.0,NaN,Blvd. Fr. Roosevelt,24.0,Wallonia,Hainaut (le),Soignies,Soignies,NaN,7060.0,Yes,NaN,NaN
1,70.0,NaN,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,NaN,4630.0,NaN,NaN,NaN
2,70.0,NaN,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,NaN,4630.0,NaN,NaN,NaN
3,71.0,NaN,Place Saint - Lambert,NaN,Wallonia,Liège,Liège,Liège,NaN,4020.0,NaN,NaN,NaN
4,72.0,NaN,Rue Du Doyard,NaN,Wallonia,Liège,Verviers,Lierneux,NaN,4990.0,NaN,NaN,NaN


In [ ]:
# CLEANING AND ADJUSTING AED DATAFRAME

## - NOT USE ANYMORE, SOLVING OF PREVIOUS ISSUES REMOVE THIS REQUIREMENT
# # 1. Clean wierd characters

# aedClean_df = aed_df
# column_names = list(aedClean_df.columns.values)
# # print(column_names)
# for name in column_names:
#     if isinstance(aedClean_df[name][1], str): #not robust if value of row 1 is missing!
#         print(name)
#         aedClean_df[name] = aedClean_df[name].str.replace('Ã¨','è')
#         aedClean_df[name] = aedClean_df[name].str.replace('Ã©','é')
#         aedClean_df[name] = aedClean_df[name].str.replace('Ã§','ç')
#         aedClean_df[name] = aedClean_df[name].str.replace('Ã´','ô')
#         aedClean_df[name] = aedClean_df[name].str.replace('Av.','Avenue')
#         aedClean_df[name] = aedClean_df[name].str.replace('Bd.','Boulevard')
#         aedClean_df[name] = aedClean_df[name].str.replace('Blvd.','Boulevard')

In [18]:
aed_df.dtypes

id                float64
type               object
address            object
number            float64
Region             object
Province           object
Arrondissement     object
Municipality       object
type.1             object
postal_code       float64
public             object
available          object
hours              object
dtype: object

In [20]:
#  2. Add numberStr column which return string format of the integer of the street number
import math

def streetnumber(number):
    """ 
    Return string format of the input

    Input:
        number = street number [float]
    Output:
        string of the street number [string]
    """
    if not math.isnan(number):
        newnumber = str(int(number))
    else:
        newnumber = ''
    return newnumber

aed_df['numberStr'] = aed_df['number'].apply(streetnumber)
aed_df.head(5)

,id,type,address,number,Region,Province,Arrondissement,Municipality,type.1,postal_code,public,available,hours,numberStr
0,13.0,NaN,Blvd. Fr. Roosevelt,24.0,Wallonia,Hainaut (le),Soignies,Soignies,NaN,7060.0,Yes,NaN,NaN,24
1,70.0,NaN,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,NaN,4630.0,NaN,NaN,NaN,76
2,70.0,NaN,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,NaN,4630.0,NaN,NaN,NaN,76
3,71.0,NaN,Place Saint - Lambert,NaN,Wallonia,Liège,Liège,Liège,NaN,4020.0,NaN,NaN,NaN,
4,72.0,NaN,Rue Du Doyard,NaN,Wallonia,Liège,Verviers,Lierneux,NaN,4990.0,NaN,NaN,NaN,


In [22]:
# FUNCTION ALSO DEFINE IN THE MODULE 'network_analysis_module'

# ADD LOCATION COORDONATE TO AED DATAFRAME

# using geopy library and Nominatim class object
# NOTE: Take long times to run (~10 min for 160 demands !)
# Good practice to save dataframe into table !

from geopy.geocoders import Nominatim
def locationBE(numberStr, address, Arrondissement):
    """
    Convert address into coordonate

    Input : 
        numberStr = number of house on the street [string],
        address = name of the street [string],
        Arrondissement = municipality [string]

    Output:
        latitude,longitude [float,float]
    """
    try:
        geolocator = Nominatim(user_agent="my_user_agent")
        street = numberStr+' '+address
        city = Arrondissement
        country = 'Belgique'
        loc = geolocator.geocode(street+','+city+','+ country, timeout=120)
        return loc.latitude,loc.longitude
    except:
        return float('nan'),float('nan')

# Testing locationBE function
# lat,lon = locationBE('1','Albert Leemansplein','Bruxelles')
# print(lat)

In [ ]:
# ISSUE: too much demand to geopy -> take too long time
# SOLUTION : proceed by part, see next code section

# # AED multipoint
# # Take more than 1 hour
# aed_df['Latitude','Longitude'] = aed_df.apply(lambda x: locationBE(x['numberStr'], x['address'], x['Arrondissement']), axis=1)
# aedOK_df = aed_df[['numberStr','address','Arrondissement','Latitude','Longitude']]

In [ ]:
#AED multipoint
max = 16600 #then do the last 68
n = 166

for i in range(1, 45):
    aedpart_df=aed_df.loc[(i*166):(i+1)*166]
    aedpart_df['Latitude','Longitude'] = aedpart_df.apply(lambda x: locationBE(x['numberStr'], x['address'], x['Arrondissement']), axis=1)
    aedpart_df.to_csv('aedOK'+str(i)+'.csv', index=False)

# Remove data from part 42 to 46 due to issue with geopy
# There is an issue with data from head(42 to 44)

for i in range(45, 166):
    aedpart_df=aed_df.loc[(i*166):(i+1)*166]
    aedpart_df['Latitude','Longitude'] = aedpart_df.apply(lambda x: locationBE(x['numberStr'], x['address'], x['Arrondissement']), axis=1)
    aedpart_df.to_csv('aedOK'+str(i)+'.csv', index=False)

    

In [ ]:
aedOK1_df = pd.read_csv('aedOK1.csv', sep=',')
aedOK1_df.head(5)

In [ ]:
# concate the part togheter

aed_list = []
for i in range(106): #I remove the 165 because
    if i >= 42 and i <=44:
        continue
    else:
        exec("aedOK"+str(i)+"_df = pd.read_csv('aedOK"+str(i)+".csv', sep=',')")
        eval("aed_list.append(aedOK"+str(i)+"_df)")

aedCoordonate = pd.concat(aed_list)
aedCoordonate.head(5)
# aedCoordonate.to_csv('aedCoordonate.csv', index=False)


In [24]:
aedCoordonate = pd.read_csv('aedCoordonate.csv', sep=',')
aedCoordonate.head(5)

,id,type,address,number,Region,Province,Arrondissement,Municipality,type.1,postal_code,public,available,hours,numberStr,"('Latitude', 'Longitude')"
0,13.0,NaN,Boulevard Fr. Roosevelt,24.0,Wallonia,Hainaut (le),Soignies,Soignies,NaN,7060.0,Yes,NaN,NaN,24.0,"(nan, nan)"
1,70.0,NaN,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,NaN,4630.0,NaN,NaN,NaN,76.0,"(nan, nan)"
2,70.0,NaN,Ch. De Wégimont,76.0,Wallonia,Liège,Liège,Soumagne,NaN,4630.0,NaN,NaN,NaN,76.0,"(nan, nan)"
3,71.0,NaN,Place Saint - Lambert,NaN,Wallonia,Liège,Liège,Liège,NaN,4020.0,NaN,NaN,NaN,NaN,"(50.6451853, 5.5742929)"
4,72.0,NaN,Rue Du Doyard,NaN,Wallonia,Liège,Verviers,Lierneux,NaN,4990.0,NaN,NaN,NaN,NaN,"(50.2856407, 5.7808696)"
